### <center>__将各个大洋插值到指定网格__

grid_path掩膜原始格点，datapath目标网格格点

In [1]:
output_photo           = "/home/yyq/mls/dataprocess/grid/grid"
outfile                = "/home/yyq/mls/dataprocess/grid/basin_new.nc"
grid_path              = "/home/yyq/mls/dataprocess/grid/BASIN_362X196_new.nc"
datapath               = "/data06/yyq/mls/licom2/addwater_new/exe/"

In [2]:
f_grid                 = addfile(grid_path, "r")
f_year                 = addfile(datapath+"YME.nc", "r")

In [3]:
lat_new                = f_year->lat
lon_new                = f_year->lon({0:359})  
printVarSummary(lat_new)
printVarSummary(lon_new)

Variable: lat_new
Type: float
Total Size: 460 bytes
            115 values
Number of Dimensions: 1
Dimensions and sizes:	[lat | 115]
Coordinates: 
            lat: [90..-78]
Number Of Attributes: 4
  standard_name :	latitude
  long_name :	latitude (on T grids)
  units :	degrees_north
  axis :	YVariable: lon_new
Type: float
Total Size: 720 bytes
            180 values
Number of Dimensions: 1
Dimensions and sizes:	[lon | 180]
Coordinates: 
            lon: [ 0..358]
Number Of Attributes: 4
  standard_name :	longitude
  long_name :	longitude (on T grids)
  units :	degrees_east
  axis :	X

In [4]:
indd                   = (/f_grid->ind(:,{0:359})/)
lat                    = (/f_grid->lat/)
lon                    = (/f_grid->lon({0:359})/)  
printVarSummary(lat)
printVarSummary(lon)

Variable: lat
Type: float
Total Size: 784 bytes
            196 values
Number of Dimensions: 1
Dimensions and sizes:	[196]
Coordinates: 
Number Of Attributes: 1
  _FillValue :	nanVariable: lon
Type: float
Total Size: 1440 bytes
            360 values
Number of Dimensions: 1
Dimensions and sizes:	[360]
Coordinates: 
Number Of Attributes: 1
  _FillValue :	nan

整个大西洋赋值2，印度洋3，太平洋4

In [5]:
indd                   = where(indd.eq.5,2,indd)
indd                   = where(indd.eq.7,4,indd)
indd                   = where(indd.eq.6,3,indd)

插值

In [6]:
indd_new               = linint2 (lon,lat,indd, True, lon_new,lat_new, 0)

写入metadata

In [7]:
indd_new!0             = "lat"                                                            
indd_new!1             = "lon"
lon_pts                = lon_new
lat_pts                = lat_new
lon_pts@units          = "degrees_east" 
lat_pts@units          = "degrees_north"
indd_new&lon           = lon_pts 
indd_new&lat           = lat_pts
printVarSummary(indd_new)

Variable: indd_new
Type: float
Total Size: 82800 bytes
            20700 values
Number of Dimensions: 2
Dimensions and sizes:	[lat | 115] x [lon | 180]
Coordinates: 
            lat: [90..-78]
            lon: [ 0..358]
Number Of Attributes: 1
  _FillValue :	9.96921e+36

输出nc文件

In [8]:
if (isfilepresent(outfile)) then
    system ("rm " + outfile)
end if
f_output = addfile(outfile, "c")
f_output->lat = lat_new
f_output->lon = lon_new
f_output->indd = indd_new

;画图检验

In [9]:
wks      = gsn_open_wks("png", output_photo)
plots    = new(1,"graphic")
res_0    = True
    res_0@gsnDraw                   =  False
    res_0@gsnFrame                  =  False
    res_0@cnFillOn                  =  True
    res_0@cnInfoLabelOn             =  False
    res_0@cnLinesOn                 =  False
    res_0@cnLineLabelsOn            =  False
    res_0@lbLabelBarOn              =  False
    res_0@mpCenterLonF              = 180
plots(0) = gsn_csm_contour_map(wks,indd_new, res_0)

resP    = True
    resP@gsnPanelLabelBar           = False
    resP@lbLabelAutoStride          = False
    resP@lbBoxSeparatorLinesOn      = False
    resP@lbBoxLinesOn               = False
    resP@lbBoxEndCapStyle           = "TriangleBothEnds"
gsn_panel(wks, plots, (/1,1/), resP)

![grid](./grid.png)